## 0. 필요 모듈 및 데이터 로드

In [113]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from tqdm.notebook import tqdm

In [220]:
train = pd.read_csv('데이터/train_v2.csv')
test = pd.read_csv('데이터/test_v2.csv')
submission = pd.read_csv('데이터/extractive_sample_submission_v2.csv')

print(train.shape)
print(test.shape)
print(submission.shape)

(42803, 5)
(9987, 3)
(9987, 2)


## 1. 클래스 정의

#### 1.1 텍스트 전처리 및 토크나이징

In [116]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리",
                          "동아일보", "중앙일보", "조선일보", "기자", "아", "휴", "아이구",
                          "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해",
                          "을", "를", "에", "의", "가"]

        
    def text2sentences(self, text):
        sentences = text[2:-2].split("', '")
        return sentences
    

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
            if noun not in self.stopwords and len(noun) > 1]))
        return nouns

#### 1.2 tf-idf 모델 적용 및 그래프

In [117]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []

    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence

    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

#### 1.3 텍스트 랭크 계산

In [118]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

#### 1.4 텍스트랭크 적용 및 추출요약

In [136]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)

        self.nouns = self.sent_tokenize.get_nouns(self.sentences)

        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def summarize_index(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
  
        index.sort()
        for idx in index:
            summary.append(idx)
        return summary
    
    def summarize_sentence(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
  
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords

In [69]:
def result_index(dfm):
    textrank_index_list = []

    for i in dfm:
        textrank = TextRank(i)
        textrank_index = textrank.summarize(3)
        textrank_index_list.append(textrank_index)
        
    return textrank_index_list

%time result_col = result(train['article_original'])
result_col[:30]

KeyboardInterrupt: 

[[0, 1, 7],
 [3, 9, 19],
 [0, 5, 6],
 [5, 6, 9],
 [2, 4, 9],
 [0, 3, 4],
 [1, 2, 4],
 [2, 3, 5],
 [0, 2, 7],
 [0, 3, 13],
 [10, 14, 19],
 [2, 3, 8],
 [1, 2, 5],
 [1, 5, 7],
 [1, 9, 12],
 [2, 8, 10],
 [0, 1, 3],
 [3, 4, 7],
 [3, 8, 11],
 [7, 9, 11],
 [2, 3, 16],
 [1, 2, 7],
 [1, 4, 7],
 [0, 1, 5],
 [1, 2, 6],
 [0, 6, 8],
 [0, 2, 7],
 [0, 1, 4],
 [0, 4, 5],
 [2, 4, 5]]

In [158]:
def result_sentence(dfm):

    text_list = []
    for i in dfm['article_original']:
        textrank = TextRank(i)

        textrank_list = []
        for row in textrank.summarize_sentence(3):
            textrank_list.append(row)
        textrank_sentence = '\n'.join(textrank_list)
        text_list.append(textrank_sentence)
    return text_list

%time result_sentence = result_sentence(test)

Wall time: 11min 9s


In [221]:
submission = pd.read_csv('데이터/extractive_sample_submission_v2.csv')
submission2 = pd.read_csv('데이터/extractive_sample_submission_v2.csv')
submission['summary'] = result_sentence

submission.to_csv('submission2.csv', index=False)
print(pd.read_csv('submission2.csv'))
print(submission2)

             id                                            summary
0     500733466  ▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓...
1     500742482  벌떼해장국의 주 메뉴는 뼈해장국과 감자탕이다.\n또한 벌떼해장국의 추천 메뉴는 콩나...
2     500742484  반면 면천면 휘발유 평균 가격은 1478원으로 가장 낮았으며, 경유는 정미면이 평균...
3     504213810  어기구 국회의원이 천연가스의 안정적 수급을 위해 2020년 착공이 예정돼 있는 액화...
4     505279620  그러나 지난해 정책자문위원회 소집횟수는 전체회의 1회에 그치고 있으며, 위원들의 의...
...         ...                                                ...
9982  745338220  지지부진한 인천 내항 재개발사업의 물꼬를 터 줄 것으로 기대했던 도시재생 혁신지구 ...
9983  745367988  청와대는 30일 문재인 대통령의 ‘1호 공약’인 고위공직자범죄수사처(공수처) 설치법...
9984  745368130  광주지역 광공업 생산 감소율이 14개월 만에 최고를 기록했다.\n광주 감소율은 지난...
9985  745368136  아름다운 가게 용봉점 헌책방이 개점 10년만에 문을 닫는 다.\n헌책방은 매년 기증...
9986  745368284  현대차그룹은 최근 한국렌터카사업조합연합회와 ‘미래 모빌리티 사업 협력을 위한 양해각...

[9987 rows x 2 columns]
             id              summary
0     500733466  추출요약1\n추출요약2\n추출요약3
1     500742482  추출요약1\n추출요약2\n추출요약3
2     500742484  추출요약1\n추출요약2\n추출요약3
3     504213810  추출요약1\

In [223]:
test['article_original'][1]

"['신 벌떼해장국이 손님들의 성원에 보답하고자 24시간 영업을 재개한다.', '또한 이와 함께 오는 12일부터 31일까지 특별 감사 이벤트를 실시할 예정이다.', '이 기간 동안 야간(저녁 11시~오전 7시)에 찾는 손님들을 위해 뼈해장국은 2000원, 감자탕은 5000원 할인된 가격으로 제공한다.', '남기순 대표는 “잊지 않고 꾸준히 찾아 주신 손님들의 감사에 보답하고자 24시 영업을 재개하고 작은 이벤트를 마련했다”고 말했다.', '벌떼해장국의 주 메뉴는 뼈해장국과 감자탕이다.', '돼지 목뼈만을 사용해 부드러운 살코기가 뼈에 가득 붙어 있다.', '또한 육수는 초벌로 삶은 뼈를 한 번 더 4시간 동안 푹 고아낸 것을 사용해 깊고 진한 맛이 특징이다.', '우릴 때는 무와 양파, 양파 껍질 및 대파 뿌리 등이 들어가 잡냄새가 없으며 이밖에도 국내산 마늘과 생강 등을 넣어 조미료 없이 감칠맛이 난다.', '여기에 우거지와 깻잎, 팽이버섯 등을 넣어 푸짐함을 더한다.', '또 대파와 청양고추를 넣어 매콤한 맛을 내면 한 그릇에도 든든한 뼈해장국이 완성된다.', '감자탕은 전골식으로 끓여가면서 먹을 수 있다.', '감자탕에는 생감자와 콩나물이 더해져 우러날수록 국물이 시원하다.', '한편 겨울철 특별메뉴로 굴김치해장국이 마련돼 있다.', '이미 입소문 난 메뉴로 뼈해장국 수준의 인기를 보인다고.', '굴김치해장국에는 직접 담근 묵은지를 사용해 국물 맛이 시원하다.', '여기에 통영에서 공수한 신선한 굴과 생콩나물이 들어간다.', '미리 조리해 놓는 방식이 아닌 주문 즉시 조리되기 때문에 콩나물의 아삭함이 살아있다.', '또한 벌떼해장국의 추천 메뉴는 콩나물황태해장국이다.', '황태 단가가 높음에도 벌떼해장국에서는 저렴한 가격으로 손님들에게 제공하고 있다.', '황태는 고소한 맛을 위해 들기름에 볶아 사용하며 역시 주문 즉시 만든다.', '지금은 단체 회식까지도 가능한 벌떼해장국이지만 처음 시작하던 15년 전에는 겨우 5평 남짓한 크기였다.', 'IMF 당시 

In [225]:
submission['summary'][0]

'▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.\n석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.\n게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 2년 동안의 손실 보상은 받을 수 없는 상황이다.'

In [226]:
submission2

,id,summary
0,500733466,추출요약1\n추출요약2\n추출요약3
1,500742482,추출요약1\n추출요약2\n추출요약3
2,500742484,추출요약1\n추출요약2\n추출요약3
3,504213810,추출요약1\n추출요약2\n추출요약3
4,505279620,추출요약1\n추출요약2\n추출요약3
...,...,...
9982,745338220,추출요약1\n추출요약2\n추출요약3
9983,745367988,추출요약1\n추출요약2\n추출요약3
9984,745368130,추출요약1\n추출요약2\n추출요약3
9985,745368136,추출요약1\n추출요약2\n추출요약3


In [195]:
(submission['id'] == submission2['id']).value_counts()

True    9987
Name: id, dtype: int64